# Parameter Inference

### Importing libraries

In [ ]:
import numpy as np
import pylab as pl
import pytwalk
import os
from tempfile import TemporaryFile
from scipy import integrate

### Create a new folder to save the results
if not os.path.exists("covid"):
    os.makedirs("covid")

### Importing March data

Daily data from march 10 to april 20

In [ ]:
infec_march_all = np.load('data2/infec_march.npy')
death_march_all = np.load('data2/death_march.npy')
hosp_march_all = np.load('data2/hosp_march.npy')

infec_april_all = np.load('data2/infec_april.npy')
death_april_all = np.load('data2/death_april.npy')
hosp_april_all = np.load('data2/hosp_april.npy')

infec_march = infec_march_all[9:]
infec_march1 = infec_march_all[9:25]
infec_april = infec_april_all[:10]
death_march = death_march_all[9:]
death_april = death_april_all[:10]
hosp_march = hosp_march_all[9:]
hosp_april = hosp_april_all[:10]
data_infec = np.concatenate((infec_march, infec_april))
data_death = np.concatenate((death_march, death_april))
data_hosp = np.concatenate((hosp_march, hosp_april))
datos=TemporaryFile()
np.save('covid/data_infec',data_infec)
np.save('covid/data_infec1',infec_march1)
np.save('covid/data_death',data_death)
np.save('covid/data_hosp',data_hosp)
N_days1 = len(infec_march1)
N_days = len(data_infec)
data_cumu_death = np.zeros(N_days)
data_cumu_death[0] = data_death[0]
for i in range(1,N_days):
    data_cumu_death[i] = data_death[i] + data_cumu_death[i-1]


Setting parameter values from literature

In [ ]:
### parameters first paper
gamma_H = 1.0/5.0
gamma_A = 1.0/7.0
gamma = 1.0/7.0
p = 0.5
k = 1.0/5.0

Ordinary Differential Equation System to model the dynamics

In [ ]:
### Set the dynamical system
def rhs(x,t,q):
    beta_I, eta, m, N = q
    m_I = 0.1*m
    beta_A = 2.0 * beta_I

    fx = np.zeros(7)
    fx[0] = -(beta_A*x[2]+beta_I*x[3])*x[0]/N   ### S dot
    fx[1] = (beta_A*x[2]+beta_I*x[3])*x[0]/N - k*x[1]  ### E dot
    fx[2] = (1-p)*k*x[1] -gamma_A*x[2] ### A dot
    fx[3] = p*k*x[1] - gamma*x[3] - eta*x[3] -m_I*x[3] ### I dot
    fx[4] = eta*x[3] - gamma_H*x[4] - m*x[4] ### H dot
    fx[5] = gamma_A * x[2] + gamma*x[3] + gamma_H*x[4] ### R dot
    fx[6] = m*x[4] + m_I*x[3]   ### D dot
    return fx

In [ ]:
### Set initial conditions for ODE system
A_0 = 20.0
H_0 = 0.0
R_0 = 0.0
D_0 = 0.0

Time2 = np.arange(0,N_days,1)

We compute the numerical solution of the ODE system. The variable N_days1 represent the date when the initial lockdown was implemented

In [ ]:
def sol_betas(x0,q1,q2):
    Time0 = np.arange(0,N_days1,1)
    def soln(x0,Time,q):
        return integrate.odeint(rhs,x0,Time,args=(q,))
    my_soln0 = soln(x0,Time0,q1)
    Time1 = np.arange(N_days1-1,N_days,1)
    my_soln1 = soln(my_soln0[-1],Time1,q2)
    return my_soln0, my_soln1

def log_factorial(a):
    if a==1 or a==0:
        return 0
    else:
        return np.log(a) + log_factorial(a-1)


We propose of prior distribution parameters

In [ ]:
k0 = 1.0
k1 = 20.0
k2 = 0.1
k3 = 25.0e2/49
theta0 = 1.0
theta3 = 49e3/5.0
var_deaths = 20**2

For the Monte Carlo, we determine the minus log of the posterior to run the twalk.

In [ ]:
def energy(q): # -log of the posterior
    E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = q
    N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
    q1 = np.array([beta_I1, eta, m, N])
    q2 = np.array([beta_I2, eta, m, N])

    x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])

    my_soln1, my_soln2 = sol_betas(x0,q1,q2)
    exposed_day1 = my_soln1[:,1]
    exposed_day2 = my_soln2[:,1][1:]
    exposed_day = np.concatenate((exposed_day1, exposed_day2))
    infected_day1 = my_soln1[:,3]
    infected_day2 = my_soln2[:,3][1:]
    infected_day = np.concatenate((infected_day1, infected_day2))
    cumu_death1 = my_soln1[:,6]
    cumu_death2 = my_soln2[:,6][1:]
    cumu_death = np.concatenate((cumu_death1,cumu_death2))
    incidence_cases = np.ones(len(exposed_day))
    incidence_cases[0]=p*k*exposed_day[0]
    hosp_cases = np.ones(len(infected_day))
    hosp_cases[0]=eta*infected_day[0]
    for i in np.arange(1,N_days,1):
        incidence_cases[i]=0.5*p*k*(exposed_day[i]+exposed_day[i-1])
        hosp_cases[i]= 0.5*eta*(infected_day[i]+infected_day[i-1])

    ### Poisson likelihood for new infectious
    log_likelihood_i = 0

    for i in range(N_days):
        log_likelihood_i = log_likelihood_i-incidence_cases[i]+data_infec[i]*np.log(incidence_cases[i])\
                        -log_factorial(data_infec[i])

    ### Poisson likelihood for new hospitalized
    log_likelihood_h = 0

    for i in range(N_days):
        log_likelihood_h = log_likelihood_h-hosp_cases[i]+data_hosp[i]*np.log(hosp_cases[i])\
                        -log_factorial(data_hosp[i])

    ### Gaussian likelihood for cumulative deaths
    log_likelihood_d = -0.5*np.dot(data_cumu_death-cumu_death, data_cumu_death-cumu_death)/ var_deaths

    a0 = (k1-1)*np.log(q[0])- (q[0]/theta0)   ## gamma distribution for E_0
    a1 = (k1-1)*np.log(q[1])- (q[1]/theta0)   ## gamma distribution for I_0
    a2 = (k0-1)*np.log(q[2])- (q[2]/theta0)   ## gamma distribution for beta_I1
    a3 = (k2-1)*np.log(q[3])- (q[3]/theta0)   ## gamma distribution for eta
    a4 = (k2-1)*np.log(q[4])- (q[4]/theta0)   ## gamma distribution for mu
    a5 = (k0-1)*np.log(q[5])- (q[5]/theta0)   ## gamma distribution for beta_I2
    a6 = (k3-1)*np.log(q[6])- (q[6]/theta3)   ## gamma distribution for S_0

    log_prior = a0 + a1 + a2 + a3 + a4 + a5 + a6

    return -log_likelihood_i -log_likelihood_h - log_likelihood_d - log_prior

Also, we specified the support of the parameters, the initial point of the chain, the burnin and total samples wanted

In [ ]:
def support(q):
    rt = True
    rt &= 0.0 < q[0] <250
    rt &= 0.0 < q[1] <100
    rt &= 0.0 < q[2] <10
    rt &= 0.0 < q[3] <1.0
    rt &= 0.0 < q[4] <1.0
    rt &= 0.0 < q[5] <10.0
    rt &= 1.0e5 < q[6] <2.0e6
    return rt

def init():
    q = np.zeros(7)
    q[0] = np.random.gamma(k1, theta0)
    q[1] = np.random.gamma(k1, theta0)
    q[2] = np.random.gamma(k0, theta0)
    q[3] = np.random.gamma(k2, theta0)
    q[4] = np.random.gamma(k2, theta0)
    q[5] = np.random.gamma(k0, theta0)
    q[6] = np.random.uniform(2.0e5, 3.0e5)
    return q

burnin = 7000
T = 40000

covid = pytwalk.pytwalk(n=7,U=energy,Supp=support)
y0=init()
yp0=init()
covid.Run(T,y0,yp0)


We save the information of the chain in the 'covid' folder

In [ ]:
cadena=TemporaryFile()
np.save('covid/cadena',covid.Output)

chain = covid.Output

Energy = chain[:,-1]

pl.figure()
pl.plot(range(T+1),Energy)
pl.xlabel("Number samples", fontsize=14)
pl.ylabel("Energy", fontsize=14)
pl.savefig('covid/energy.png')
pl.show()

pl.figure()
pl.plot(range(T+1-burnin),Energy[burnin:])
pl.xlabel("Number samples", fontsize=14)
pl.ylabel("Energy", fontsize=14)
pl.savefig('covid/energy_burn.png')
pl.show()

pl.figure()
pl.plot(range(T+1-burnin),Energy[burnin:])
pl.ylim(min(Energy)-50,min(Energy)+100)
pl.xlabel("Number samples", fontsize=14)
pl.ylabel("Energy", fontsize=14)
pl.savefig('covid/energy_burn_limit.png')
pl.show()


We compute the posterior estimates: the Maximum a Posterior (MAP) and the posterior mean 

In [ ]:
### Computing the MAP estimate
energy_MAP = min(Energy)
loc_MAP = np.where(Energy==energy_MAP)[0]
MAP = chain[loc_MAP[-1]]
MAP = MAP[:-1]

### Computing the posterior mean
Post_mean = np.ones(7)
Post_mean[0] = np.mean(chain[burnin:,0])
Post_mean[1] = np.mean(chain[burnin:,1])
Post_mean[2] = np.mean(chain[burnin:,2])
Post_mean[3] = np.mean(chain[burnin:,3])
Post_mean[4] = np.mean(chain[burnin:,4])
Post_mean[5] = np.mean(chain[burnin:,5])
Post_mean[6] = np.mean(chain[burnin:,6])


Marginal Posterior histograms

In [ ]:
pl.figure()
pl.hist(chain[burnin:,0],bins=30)
pl.xlabel(r"$E_0$", fontsize=14)
pl.savefig('covid/E_0.png')
pl.show()

pl.figure()
pl.hist(chain[burnin:,1],bins=30)
pl.xlabel(r"$I_0$", fontsize=14)
pl.savefig('covid/I_0.png')
pl.show()

pl.figure()
pl.hist(chain[burnin:,2],bins=30)
pl.xlabel(r"$\beta_I$", fontsize=14)
pl.savefig('covid/beta_I1.png')
pl.show()

pl.figure()
pl.hist(chain[burnin:,3],bins=30)
pl.xlabel(r"$\eta$", fontsize=14)
pl.savefig('covid/eta.png')
pl.show()

pl.figure()
pl.hist(chain[burnin:,4],bins=30)
pl.xlabel(r"$m$", fontsize=14)
pl.savefig('covid/m.png')
pl.show()

pl.figure()
pl.hist(chain[burnin:,5],bins=30)
pl.xlabel(r"$\beta_I$", fontsize=14)
pl.savefig('covid/beta_I2.png')
pl.show()

pl.figure()
pl.hist(chain[burnin:,6],bins=30)
pl.xlabel(r"$S_0$", fontsize=14)
pl.savefig('covid/S_0.png')
pl.show()


In [ ]:
E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = MAP
N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
q1_MAP = np.array([beta_I1, eta, m, N])
q2_MAP = np.array([beta_I2, eta, m, N])

x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])

my_soln1_MAP, my_soln2_MAP = sol_betas(x0,q1_MAP,q2_MAP)
exposed_day1_MAP = my_soln1_MAP[:,1]
exposed_day2_MAP = my_soln2_MAP[:,1][1:]
exposed_day_MAP = np.concatenate((exposed_day1_MAP, exposed_day2_MAP))
infected_day1_MAP = my_soln1_MAP[:,3]
infected_day2_MAP = my_soln2_MAP[:,3][1:]
infected_day_MAP = np.concatenate((infected_day1_MAP, infected_day2_MAP))
cumu_death1_MAP = my_soln1_MAP[:,6]
cumu_death2_MAP = my_soln2_MAP[:,6][1:]
cumu_death_MAP = np.concatenate((cumu_death1_MAP,cumu_death2_MAP))
incidence_cases_MAP = np.ones(len(exposed_day_MAP))
incidence_cases_MAP[0]=p*k*exposed_day_MAP[0]
hosp_cases_MAP = np.ones(len(infected_day_MAP))
hosp_cases_MAP[0]=eta*infected_day_MAP[0]
for i in np.arange(1,N_days,1):
    incidence_cases_MAP[i]=0.5*p*k*(exposed_day_MAP[i]+exposed_day_MAP[i-1])
    hosp_cases_MAP[i]= 0.5*eta*(infected_day_MAP[i]+infected_day_MAP[i-1])


E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = Post_mean
N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
q1_CM = np.array([beta_I1, eta, m, N])
q2_CM = np.array([beta_I2, eta, m, N])

x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])

my_soln1_CM, my_soln2_CM = sol_betas(x0,q1_CM,q2_CM)
exposed_day1_CM = my_soln1_CM[:,1]
exposed_day2_CM = my_soln2_CM[:,1][1:]
exposed_day_CM = np.concatenate((exposed_day1_CM, exposed_day2_CM))
infected_day1_CM = my_soln1_CM[:,3]
infected_day2_CM = my_soln2_CM[:,3][1:]
infected_day_CM = np.concatenate((infected_day1_CM, infected_day2_CM))
cumu_death1_CM = my_soln1_CM[:,6]
cumu_death2_CM = my_soln2_CM[:,6][1:]
cumu_death_CM = np.concatenate((cumu_death1_CM,cumu_death2_CM))
incidence_cases_CM = np.ones(len(exposed_day_CM))
incidence_cases_CM[0]=p*k*exposed_day_CM[0]
hosp_cases_CM = np.ones(len(infected_day_CM))
hosp_cases_CM[0]=eta*infected_day_CM[0]
for i in np.arange(1,N_days,1):
    incidence_cases_CM[i]=0.5*p*k*(exposed_day_CM[i]+exposed_day_CM[i-1])
    hosp_cases_CM[i]= 0.5*eta*(infected_day_CM[i]+infected_day_CM[i-1])

pl.figure()
pl.plot(Time2,data_infec,'ro')
pl.plot(Time2,incidence_cases_MAP, label = 'MAP')
pl.plot(Time2,incidence_cases_CM, label = 'CM')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Daily infetious", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_inf_fitted.png')
pl.show()

pl.figure()
pl.plot(Time2,data_cumu_death,'ro')
pl.plot(Time2,cumu_death_MAP, label = 'MAP')
pl.plot(Time2,cumu_death_CM, label = 'CM')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Cumulative deaths", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_death_fitted.png')
pl.show()

pl.figure()
pl.plot(Time2,data_hosp,'ro')
pl.plot(Time2,hosp_cases_MAP, label = 'MAP')
pl.plot(Time2,hosp_cases_CM, label = 'CM')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Daily new hospitalizations", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_hosp_fitted.png')
pl.show()


pl.figure()
for j in range(200):
    q = chain[-1-10*j][:-1]
    E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = q
    N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
    q1 = np.array([beta_I1, eta, m, N])
    q2 = np.array([beta_I2, eta, m, N])

    x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])

    my_soln1, my_soln2 = sol_betas(x0,q1,q2)
    exposed_day1 = my_soln1[:,1]
    exposed_day2 = my_soln2[:,1][1:]
    exposed_day = np.concatenate((exposed_day1, exposed_day2))
    incidence_cases = np.ones(len(exposed_day))
    incidence_cases[0]=p*k*exposed_day[0]
    for i in np.arange(1,N_days,1):
        incidence_cases[i]=0.5*p*k*(exposed_day[i]+exposed_day[i-1])
    pl.plot(Time2,incidence_cases, 'k', color='0.75')
pl.plot(Time2,data_infec,'ro', label = 'data')
pl.plot(Time2,incidence_cases_MAP, label = 'MAP')
pl.plot(Time2,incidence_cases_CM, label = 'CM')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Daily infetious", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_inf_fitted_UQ.png')
pl.show()


pl.figure()
for j in range(200):
    q = chain[-1-10*j][:-1]
    E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = q
    N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
    q1 = np.array([beta_I1, eta, m, N])
    q2 = np.array([beta_I2, eta, m, N])

    x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])

    my_soln1, my_soln2 = sol_betas(x0,q1,q2)
    cumu_death1 = my_soln1[:,6]
    cumu_death2 = my_soln2[:,6][1:]
    cumu_death = np.concatenate((cumu_death1,cumu_death2))
    pl.plot(Time2,cumu_death, 'k', color='0.75')
pl.plot(Time2,data_cumu_death,'ro', label = 'data')
pl.plot(Time2,cumu_death_MAP, label = 'MAP')
pl.plot(Time2,cumu_death_CM, label = 'CM')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Cumulative Deaths", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_fitted_deaths_UQ.png')
pl.show()


pl.figure()
for j in range(200):
    q = chain[-1-10*j][:-1]
    E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = q
    N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
    q1 = np.array([beta_I1, eta, m, N])
    q2 = np.array([beta_I2, eta, m, N])
    x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])
    my_soln1, my_soln2 = sol_betas(x0,q1,q2)
    infected_day1 = my_soln1[:,3]
    infected_day2 = my_soln2[:,3][1:]
    infected_day = np.concatenate((infected_day1, infected_day2))
    hosp_cases = np.ones(len(infected_day))
    hosp_cases[0]=eta*infected_day[0]
    for i in np.arange(1,N_days,1):
        hosp_cases[i]= 0.5*eta*(infected_day[i]+infected_day[i-1])
    pl.plot(Time2,hosp_cases, 'k', color='0.75')
pl.plot(Time2,data_hosp,'ro', label = 'data')
pl.plot(Time2,hosp_cases_MAP, label = 'MAP')
pl.plot(Time2,hosp_cases_CM, label = 'CM')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Daily new hospitalizations", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_hosp_fitted_UQ.png')
pl.show()


In [ ]:
E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = MAP
N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
q1_MAP_pred = np.array([beta_I1, eta, m, N])
q2_MAP_pred = np.array([beta_I2, eta, m, N])

x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])

my_soln1_MAP_pred, my_soln2_MAP_pred = sol_betas(x0,q1_MAP_pred,q1_MAP_pred)
exposed_day1_MAP_pred = my_soln1_MAP_pred[:,1]
exposed_day2_MAP_pred = my_soln2_MAP_pred[:,1][1:]
exposed_day_MAP_pred = np.concatenate((exposed_day1_MAP_pred, exposed_day2_MAP_pred))
infected_day1_MAP_pred = my_soln1_MAP_pred[:,3]
infected_day2_MAP_pred = my_soln2_MAP_pred[:,3][1:]
infected_day_MAP_pred = np.concatenate((infected_day1_MAP_pred, infected_day2_MAP_pred))
cumu_death1_MAP_pred = my_soln1_MAP_pred[:,6]
cumu_death2_MAP_pred = my_soln2_MAP_pred[:,6][1:]
cumu_death_MAP_pred = np.concatenate((cumu_death1_MAP_pred,cumu_death2_MAP_pred))
incidence_cases_MAP_pred = np.ones(len(exposed_day_MAP_pred))
incidence_cases_MAP_pred[0]=p*k*exposed_day_MAP_pred[0]
hosp_cases_MAP_pred = np.ones(len(infected_day_MAP_pred))
hosp_cases_MAP_pred[0]=eta*infected_day_MAP_pred[0]
for i in np.arange(1,N_days,1):
    incidence_cases_MAP_pred[i]=0.5*p*k*(exposed_day_MAP_pred[i]+exposed_day_MAP_pred[i-1])
    hosp_cases_MAP_pred[i]= 0.5*eta*(infected_day_MAP_pred[i]+infected_day_MAP_pred[i-1])

pl.figure()
pl.plot(Time2,data_infec,'ro')
pl.plot(Time2,incidence_cases_MAP, label = 'MAP')
pl.plot(Time2,incidence_cases_MAP_pred, label = 'MAP1')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Daily infetious", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_inf_fitted_pred.png')
pl.show()

pl.figure()
pl.plot(Time2,data_hosp,'ro')
pl.plot(Time2,hosp_cases_MAP, label = 'MAP')
pl.plot(Time2,hosp_cases_MAP_pred, label = 'MAP1')
pl.xlabel("Days", fontsize=14)
pl.ylabel("Daily new hospitalizations", fontsize=14)
pl.legend(loc=0)
pl.savefig('covid/data_hosp_fitted_pred.png')
pl.show()


In [ ]:
Final_point = np.zeros([5000,7])
for j in range(5000):
    q = chain[-1-j][:-1]
    E_0, I_0, beta_I1, eta, m, beta_I2, S_0 = q
    N = S_0 + E_0 + A_0 + I_0 + H_0 + R_0 + D_0
    q1 = np.array([beta_I1, eta, m, N])
    q2 = np.array([beta_I2, eta, m, N])
    x0 = np.array([S_0, E_0, A_0, I_0, H_0, R_0, D_0])
    my_soln1, my_soln2 = sol_betas(x0,q1,q2)
    point = my_soln2[-1]
    Final_point[j] = point

pl.figure()
pl.hist(Final_point[:,0],bins=30)
pl.xlabel(r"$S_f$", fontsize=14)
pl.savefig('covid/Final_S.png')
pl.show()

pl.figure()
pl.hist(Final_point[:,1],bins=30)
pl.xlabel(r"$E_f$", fontsize=14)
pl.savefig('covid/Final_E.png')
pl.show()

pl.figure()
pl.hist(Final_point[:,2],bins=30)
pl.xlabel(r"$A_f$", fontsize=14)
pl.savefig('covid/Final_A.png')
pl.show()

pl.figure()
pl.hist(Final_point[:,3],bins=30)
pl.xlabel(r"$I_f$", fontsize=14)
pl.savefig('covid/Final_I.png')
pl.show()

pl.figure()
pl.hist(Final_point[:,4],bins=30)
pl.xlabel(r"$H_f$", fontsize=14)
pl.savefig('covid/Final_H.png')
pl.show()

pl.figure()
pl.hist(Final_point[:,5],bins=30)
pl.xlabel(r"$R_f$", fontsize=14)
pl.savefig('covid/Final_R.png')
pl.show()

pl.figure()
pl.hist(Final_point[:,6],bins=30)
pl.xlabel(r"$D_f$", fontsize=14)
pl.savefig('covid/Final_D.png')
pl.show()


mean_cond = np.mean(Final_point, axis=0)
var_cond = np.var(Final_point, axis=0)

Condiciones=TemporaryFile()
np.save('covid/mean_condiciones',mean_cond)
np.save('covid/var_condiciones',var_cond)
